In [5]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import statistics as st

def get_df(day):
    file_name = f"./round_3_data/prices_round_3_day_{day}.csv"
    df = pd.read_csv(file_name, sep=';')
    df['timestamp'] = df['timestamp'] + 1_000_000 * (day - 1)
    return df

def get_trades_df(day):
    file_name = f"./round_3_data/trades_round_3_day_{day}.csv"
    return pd.read_csv(file_name, sep=';')

def get_product(df, product):
    return df[df['product'] == product].copy()

def get_first_two_dfs():
    first_df = get_df(1)
    second_df = get_df(2)
    return pd.concat([first_df, second_df])



In [6]:
df = get_df(1)
df_trades = get_trades_df(1)

In [7]:

df_coconut = get_product(df, 'VOLCANIC_ROCK')
df_coconut_call = get_product(df, "VOLCANIC_ROCK_VOUCHER_10000")
df_coconut_call = df_coconut_call.merge(df_coconut[['timestamp', 'mid_price']], on='timestamp', suffixes=('', '_coconut'))

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_coconut['timestamp'], y=df_coconut['mid_price'], name='Coconut Mid Price'))
fig.add_trace(go.Scatter(x=df_coconut_call['timestamp'], y=df_coconut_call['mid_price'], name='Coconut Call Mid Price', yaxis='y2'))
fig.update_layout(title='Coconut and Coconut Call Prices over Timestamp', yaxis=dict(title='Coconut Mid Price'), yaxis2=dict(title='Coconut Call Bid Price', overlaying='y', side='right'))
fig.show()


In [9]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

def black_scholes_call(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot / strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    d2 = d1 - volatility * np.sqrt(time_to_expiry)
    call_price = (spot * norm.cdf(d1) - strike * norm.cdf(d2))
    return call_price

def black_scholes_put(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot / strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    d2 = d1 - volatility * np.sqrt(time_to_expiry)
    put_price = (strike * norm.cdf(-d2) - spot * norm.cdf(-d1))
    return put_price

def delta(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.cdf(d1)

def gamma(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.pdf(d1)/(spot * volatility * np.sqrt(time_to_expiry))

def vega(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.pdf(d1) * (spot * np.sqrt(time_to_expiry)) / 100

def implied_volatility(call_price, spot, strike, time_to_expiry):
    # Define the equation where the root is the implied volatility
    def equation(volatility):
        estimated_price = black_scholes_call(spot, strike, time_to_expiry, volatility)
        return estimated_price - call_price

    # Using Brent's method to find the root of the equation
    implied_vol = brentq(equation, 1e-10, 3.0, xtol=1e-10)
    return implied_vol

def realized_vol(df_coconut_call, window, step_size):
    df_coconut_call[f'log_return_{step_size}'] = np.log(df_coconut_call['mid_price_coconut'].to_numpy()/df_coconut_call['mid_price_coconut'].shift(step_size).to_numpy())
    dt = step_size / 250 / 10000 
    df_coconut_call[f'realized_vol_{step_size}'] = df_coconut_call[f'log_return_{step_size}'].rolling(window=window).apply(lambda x: np.mean(x[::step_size]**2) / dt)
    df_coconut_call[f'realized_vol_{step_size}'] = np.sqrt(df_coconut_call[f'realized_vol_{step_size}'].to_numpy())
    return df_coconut_call

def vanna(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return 1/100 * np.sqrt(time_to_expiry) * norm.pdf(d1) * (1- d1/(volatility * np.sqrt(time_to_expiry)))


In [11]:
spot_price = 10500        # Spot price of the underlying asset
strike_price = 10000      # Strike price of the option
call_price = 519         # Market price of the call option
time_to_expiry = 1      # Time to expiry in years
initial_guess = 16
df_coconut_call['time_to_expiry'] = 1 - df_coconut_call['timestamp'] / 1000000 / 7
df_coconut_call['implied_vol'] = df_coconut_call.apply(lambda row: implied_volatility(row['mid_price'], row['mid_price_coconut'], strike_price, row['time_to_expiry']), axis=1)
df_coconut_call['delta'] = df_coconut_call.apply(lambda row: delta(row['mid_price_coconut'], strike_price, row['time_to_expiry'], row['implied_vol']), axis=1)
df_coconut_call['gamma'] = df_coconut_call.apply(lambda row: gamma(row['mid_price_coconut'], strike_price, row['time_to_expiry'], row['implied_vol']), axis=1)
df_coconut_call['vega'] = df_coconut_call.apply(lambda row: vega(row['mid_price_coconut'], strike_price, row['time_to_expiry'], row['implied_vol']), axis=1)

In [12]:
df_coconut_call['cash_delta'] = df_coconut_call['delta'].to_numpy() * df_coconut_call['mid_price_coconut'].to_numpy()


In [16]:
import plotly.express as px

fig = px.line(df_coconut_call, x='timestamp', y='delta', title='Delta over Time')
fig.show()


In [17]:
fig = px.line(df_coconut_call, x='timestamp', y='vega', title='Vega over Time')
fig.show()

In [14]:
fig = px.line(df_coconut_call, x='timestamp', y='implied_vol', title='Implied Volatility over Time')
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_coconut_call['timestamp'], y=df_coconut_call['implied_vol'], name="Implied Volatility"))
fig.add_trace(go.Scatter(x=df_coconut_call['timestamp'], y=df_coconut_call['mid_price_coconut'], name="Coconut Mid Price", yaxis="y2"))
fig.update_xaxes(title_text="Timestamp")
fig.update_layout(title_text="Implied Volatility and Coconut Mid Price over Time")
fig.update_layout(yaxis2=dict(overlaying="y", side="right"))
fig.show()



In [18]:
df_coconut_call['cash_gamma'] = df_coconut_call['gamma'].to_numpy() * df_coconut_call['mid_price_coconut'].to_numpy()
df_coconut_call['gamma_1pc'] = 0.01 * df_coconut_call['cash_gamma']
fig = px.line(df_coconut_call, x='timestamp', y='cash_gamma', title='Cash Gamma over Time')
fig.show()


In [19]:
df_coconut_call['vanna'] = df_coconut_call.apply(lambda row: vanna(row['mid_price_coconut'], strike_price, row['time_to_expiry'], row['implied_vol']), axis=1)
fig = px.line(df_coconut_call, x='timestamp', y='vanna', title='Vanna over Time')
fig.show()

In [20]:
implied_vol_mean = df_coconut_call['implied_vol'].mean()
print("Implied Volatility Mean: ", implied_vol_mean)
implied_vol_std = df_coconut_call['implied_vol'].std()
print("Implied Volatility Standard Deviation: ", implied_vol_std)


Implied Volatility Mean:  0.023052698192764183
Implied Volatility Standard Deviation:  0.0012319387731193068


In [48]:
cash = 0
futures = 0
call = 0
size = 600
pnl = []
thresh = implied_vol_std
for index, row in df_coconut_call.iterrows():
    # Your code here
    implied_vol = row['implied_vol']
    call_price = row['mid_price']
    future_price = row['mid_price_coconut']
    delta = row['delta']
    if implied_vol > implied_vol_mean + implied_vol_std:
        cash = cash - (-size -  call) * call_price
        cash = cash - (size * delta - futures) * future_price
        call = -size
        futures = size * delta
    elif implied_vol < implied_vol_mean - implied_vol_std:
        cash = cash - (size -  call) * call_price
        cash = cash - (-size * delta - futures) * future_price
        call = size
        futures = -size * delta
    
    pnl.append(cash + call * call_price + futures * future_price)
        

In [49]:
fig = px.line(x=df_coconut_call['timestamp'], y=pnl, title='PnL over Time')
fig.show()


In [307]:
df_trades = get_trades_df(1)
df_trades_coupon = df_trades[df_trades['symbol'] == 'COCONUT_COUPON']
df_trades_coupon['quantity'] = df_trades_coupon['quantity'].astype(int)
df_trades_coupon = df_trades_coupon.groupby('timestamp')['quantity'].sum().reset_index()
fig = px.histogram(df_trades_coupon, x='quantity', title='Traded Volume over Time')
fig.show()
df_trades_coupon[df_trades_coupon['quantity'] > 25]



/var/folders/9b/84mpgsld28b38s8hfxg9c3rm0000gn/T/ipykernel_8547/1281816314.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,timestamp,quantity
24,26600,27
43,44600,32
77,74800,28
107,101500,29
124,124800,31
156,166800,27
166,174900,30
310,319400,33
340,352800,35
394,409600,32
